In [12]:
import scipy.io as sio
import numpy as np
import random
import matplotlib.pyplot as plt

In [13]:
#load
data = sio.loadmat('H:/Jupyter_Notebook_Workspace/AI_cw/xy.mat')

In [14]:
type(data['xy'])

numpy.ndarray

# GA Part

In [15]:
a=[[3,9],[4,5],[2,3]]
b=np.sum(a,axis=1)
c=b/sum(b)
c.cumsum()

array([0.46153846, 0.80769231, 1.        ])

In [16]:
class GA(object):
    def __init__(self,data,selectType,crossType,mutateType,populationNum,populationSize):
        self.data = data
        self.selectType = selectType # 1.Tournament 2.Roulette Wheel Selection
        self.crossType = crossType # 1.Order-based; 2.2-point Crossover
        self.mutateType = mutateType # 1.Swap; 2.Flip; 3.Slide Mutation
        self.populationNum = populationNum # The number of population
        self.populationSize = populationSize # Number of genes in chromesome
        
        self.crossRate = 0.8
        self.mutateRate = 0.2
        self.fitness = []
        self.population = []
        self.bestPath = []
        self.shuffeGenes()
        
    
    # Generate the chromesomes
    def shuffeGenes(self):
        for i in range(self.populationNum):
            gene = list(range(self.populationSize))
            np.random.shuffle(gene)
            self.population.append(gene)
        
    
    # Get the fitness of each chromesome
    def getFitness(self):
        fitness=[]
        for j in range(self.populationNum):
            fitScore = 0
            for i in range(self.populationSize):
                cA,cB = self.data[self.population[j][i-1]],self.data[self.population[j][i]]
                fitScore += np.linalg.norm(np.array(cA)-np.array(cB))
            fitness.append(fitScore)
            
        self.fitness = fitness
        self.bestScore = self.population[fitness.index(min(fitness))]
        
    
    def selection(self):
        gene = []
        population = self.population
        fitness = self.fitness

        # Tournament Selection
        if self.selectType == 1:
            # Random choose 2 candadtes
            gpoint1 = random.randint(1,self.populationNum-1)
            gpoint2 = random.randint(1,self.populationNum-1)
            while(gpoint2 != gpoint1):
                gpoint2 = random.randint(1,self.populationNum-1)

            # Compare chromesomes
            fit1 = fitness[gpoint1]
            fit2 = fitness[gpoint2]
            if (fit1>fit2): 
                gene=population[gpoint1]
            else:
                gene=population[gpoint2]               
                
        # Roulette Wheel Selection
        elif self.selectType == 2:
            index = 0
            randProb = random.random()
            
            weight = fitness/sum(fitness) # Get each probability
            accumulation = weight.cumsum() # Get the cumulative probability           
            
            for prob in accumulation:
                if prob > randProb:
                    gene = population[index]
                    break
                index += 1                    
        else:
            raise Exception('UNKNOW SELECTION TYPE!')
        
        return gene
 
    
    # Cross over 2 genes
    def crossover(self,geneA,geneB):
        gene = []
        # Random choose 2 cut points
        cpoint1 = random.randint(1,self.populationSize-2) 
        cpoint2 = random.randint(cpoint1+1,self.populationSize-1)
            
        # Order-based Crossover
        if self.crossType == 1:
            # Cut slices
            keepA = geneA[cpoint1:cpoint2]
            keepB = geneB[cpoint1:cpoint2]
            
            # Remainders after cut
            pice1 = [x for x in geneA if x not in keepB]
            pice2 = [x for x in geneB if x not in keepA]
            
            # Combine to new genes
            gene1 = pice2[:cpoint1]+keepA+pice2[cpoint1:] 
            gene2 = pice1[:cpoint1]+keepB+pice1[cpoint1:]
        
        # 2-point Crossover
        elif self.crossType ==2:
            gene1 = geneA[:cpoint1]+geneB[cpoint1:cpoint2]+geneA[cpoint2:]
            gene2 = geneB[:cpoint1]+geneA[cpoint1:cpoint2]+geneB[cpoint2:]
        else:
            raise Exception('UNKNOW CROSSOVER TYPE!')
        
        return gene1,gene2

    
    # Mutate the gene
    def mutation(self,geneM):
        gene = []
        # Random choose 2 positions
        mpoint1 = random.randint(1,self.populationSize-3)
        mpoint2 = random.randint(mpoint1+1,self.populationSize-1)
        midPice = geneM[mpoint1:mpoint2]
        
        # Swap Mutation
        if self.mutateType == 1:
            # Get the value of each point
            p1 = geneM[mpoint1]
            p2 = geneM[mpoint2]
            
            # Exchange 2 points
            geneM[mpoint1] = p2
            geneM[mpoint2] = p1
            gene = geneM
            
        # Filp Mutation 
        elif self.mutateType == 2:
            gene = geneM[:mpoint1]+list(reversed(midPice))+geneM[mpoint2:]
            
        # Slide Mutation
        elif self.mutateType == 3:
            # Choice slide point
            sPostion = random.randint(1,len(midPice))
            # Slide the midPice
            slidePice = midPice[sPostion:] + midPice[:sPostion]
            gene = geneM[:mpoint1]+slidePice+geneM[mpoint2:]
        else:
            raise Exception('UNKNOW MUTATION TYPE!')
        
        return gene
    
    
    # Get the 2 elites
    def getElite(self):
        maxIndex = []
        elitePopulation = []
        fitness = self.fitness
        
        # Get max 2 fitness in population
        for i in range(2):
            maxIndex.append(fitness.index(min(fitness)))
            fitness[fitness.index(min(fitness))]=1000000
        maxIndex.sort()
        
        elitePopulation.append(self.population[maxIndex[0]])
        elitePopulation.append(self.population[maxIndex[1]])

        return elitePopulation
    
    
    # The population multiply
    def multiply(self):
        count = 0
        newPopulation = []
        self.getFitness()
        elitePopulation = self.getElite()

        while (count < self.populationNum/2-1):
            # Selection
            geneA = self.selection()
            geneB = self.selection()
            # Crossover
            if random.random() < self.crossRate:
                geneA,geneB = self.crossover(geneA,geneB) 

            # Mutation    
            if random.random() < self.mutateRate:
                geneA = self.mutation(geneA)
                geneB = self.mutation(geneB)
                   
            newPopulation.append(geneA)
            newPopulation.append(geneB)
            count += 1 
            
        self.population = newPopulation
        self.population += elitePopulation

# TSP Part

In [31]:
# Set the parameters
populationNum = 100
populationSize = 100

# Reset the GA class
ga = GA(data=data['xy'],
        selectType=1,
        crossType=1,
        mutateType=2,
        populationNum=populationNum,
        populationSize=populationSize)

# Get distance of each path
def getDistance(path):
    distance = 0
    for i in range(populationSize):
        cA,cB = data['xy'][path[i-1]],data['xy'][path[i]]
        distance += np.linalg.norm(np.array(cA)-np.array(cB))
    return distance

# Run TSP function    
def runFunctions(gen):
    distance_list=[]
    for i in range(gen):
        ga.multiply()
        distance = getDistance(ga.bestScore)
        distance_list.append(distance)
            
    return distance,ga.bestScore,distance_list

# Run functions

In [32]:
Min_Path,BestPath,distance_list=runFunctions(3000)

In [33]:
Min_Path

218.0571363355043